In [ ]:
import os
import glob

import numpy as np
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

import ray

In [ ]:
LuftFileList = glob.glob("../DATA/MWIS_2023_NDJF_OBS/*MARWIS*csv")
VaisalaFileList = glob.glob("../DATA/MWIS_2023_NDJF_OBS/*move4*csv")

In [ ]:
luft_total_list = []
for luft in LuftFileList:
    luft_df = pd.read_csv(luft, low_memory=False)
    luft_total_list.append(luft_df)
luft_total_df = pd.concat(luft_total_list)

vi_total_list = []
for vi in VaisalaFileList:
    vi_df = pd.read_csv(vi, low_memory=False)
    vi_total_list.append(vi_df)
vi_total_df = pd.concat(vi_total_list)

drop_column_list = ['gdirection1','gdirection2','digitalcomp_x','digitalcomp_y','digitalcomp_z',
                    'ta2','rh2','loggerta', 'batteryvolt', 'rev1', 'rev2']
vi_total_df.drop(drop_column_list, axis = 1, inplace=True)



In [ ]:
vi_total_df['TIMESTAMP'] = 

,vehicle,gdate,gtime,glatitude,glongitude,galtitude,vsp,rws,rwd,tws,...,sw_up,lw_down,lw_up,netradiation,s_ta,s_status,grip,waterlayer,icelayer,snowlayer
0,A,231117,65401,37.516632,127.439939,55.2,0.0,0.5,336.00,0.49,...,4,231,308,-81,0.3,1,0.81,0.00,0.00,0.00
1,A,231117,65402,37.516632,127.439939,55.2,0.0,0.5,337.00,0.46,...,4,231,307,-81,0.3,1,0.81,0.00,0.00,0.00
2,A,231117,65403,37.516632,127.439939,55.2,0.0,0.4,334.00,0.39,...,4,231,307,-81,0.3,1,0.81,0.00,0.00,0.00
3,A,231117,65405,37.516633,127.439938,55.4,0.0,0.3,328.00,0.33,...,4,231,307,-81,0.3,1,0.81,0.00,0.00,0.00
4,A,231117,65406,37.516633,127.439938,55.4,0.0,0.3,317.00,0.32,...,4,231,307,-81,0.3,1,0.81,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10493,A,240223,124001,37.460904,126.890301,21.0,11.2,11.4,357.00,4.17,...,80,286,348,561,16.1,1,0.82,0.00,0.00,0.00
10494,A,240223,124002,37.461070,126.890166,20.2,10.9,11.5,357.00,4.17,...,82,286,348,564,16.5,1,0.82,0.00,0.00,0.00
10495,A,240223,124003,37.461148,126.890090,20.6,10.4,11.5,357.00,4.00,...,83,286,348,568,16.1,1,0.82,0.00,0.00,0.00
10496,A,240223,124004,37.461231,126.890008,23.1,11.1,11.4,356.00,4.13,...,83,286,348,553,9.3,1,0.82,0.00,0.00,0.00


In [ ]:
luft_total_df

,TIMESTAMP,YYMMDD,HHMMSS,R_Tem,R_Dew,R_Hum,R_WH,R_Sta,R_Ice,R_Fric,R_Sta_txt,BattV,PTemp
0,2023-11-17 06:54:01,231117,65400,-0.323289,-1.210844,92.73611,0.041476,5,0.000000e+00,0.808356,Chemically wet,11.99,17.76652
1,2023-11-17 06:54:02,231117,65401,-0.364372,-1.210844,93.70230,0.039128,5,0.000000e+00,0.808801,Chemically wet,12.00,17.76652
2,2023-11-17 06:54:03,231117,65402,-0.282198,-1.219853,93.95007,0.041893,5,0.000000e+00,0.808278,Chemically wet,12.00,17.76652
3,2023-11-17 06:54:04,231117,65403,-0.405463,-1.219853,93.49349,0.041732,5,0.000000e+00,0.808308,Chemically wet,12.00,17.76652
4,2023-11-17 06:54:05,231117,65404,-0.237178,-1.210038,94.23958,0.040386,5,0.000000e+00,0.808561,Chemically wet,11.99,17.79138
...,...,...,...,...,...,...,...,...,...,...,...,...,...
14796,2024-02-23 12:39:57,240223,123956,14.997340,-2.121648,30.65873,0.028232,1,3.082857e-44,0.811037,Damp,11.93,26.96393
14797,2024-02-23 12:39:58,240223,123957,14.918720,-2.092046,30.85942,0.026249,1,3.082857e-44,0.811494,Damp,11.93,26.96393
14798,2024-02-23 12:39:59,240223,123958,14.952900,-2.092046,31.13237,0.024621,1,3.082857e-44,0.811860,Damp,11.93,26.96393
14799,2024-02-23 12:40:00,240223,123959,15.024190,-2.072454,30.88432,0.020974,1,3.082857e-44,0.812761,Damp,11.93,26.96393


In [ ]:
luft_total_df.to_csv("../DATA/CONCAT/luft_2023_2024.csv", index=None)
vi_total_df.to_csv("../DATA/CONCAT/visala_2023_2024.csv", index=None)